# Tema 2. Análisis de secuencias

## Mapeo de secuencias cortas a un genoma de referencia

El objetivo del primer ejercicio es mapear un conjunto de secuencias cortas a un genoma de referencia.

Las secuencias cortas provienen de un experimento de secuenciación de un aisalado de coronavirus. Se encuentran en el archivo `data/ERR4423464.fastq`, que es un archivo de texto con formato **fastq**. Primero, échale un vistazo al archivo. Deberías poder observar algo así:

    @ERR4423464.1 M00289:305:000000000-J7B3T:1:1101:18085:1894 length=218
    CTGCTACACGCTGCGAAGCTCCCAATTTGTAATAAGAAAGCGTTCGTGATGTAGCAACAGTGATTTCTTTAGGCAGGTCCTTGATGTCACAGCGTCCTAGATGGTGTTCGAACACCATCTAGGACGCTGTGACATCAAGGACCTGCCTAAAGAAATCACTGTTGCTACATCACGAACGCTTTCTTATTACAAATTGGGAGCTTCGCAGCGTGTAGCAG
    +ERR4423464.1 M00289:305:000000000-J7B3T:1:1101:18085:1894 length=218
    BC@CCDE9F-@C7C+++6@CEFF8,CFFCFA9F9,,,,,,C:FFFCF@,C9F99C,6C8<C9,EFFFFEE9,,9,,+:9CFF9,C,FFDFFCFEGGGGGFFGGGGFFGG--,86C@8CFF9,,,;+@B;C,,C9EE,,,,,;CE,CCEA8,,,6,CE,CF9FE9CF9F9;C,C:::C7BFCDFEE9CFAE99,F@,6,,,8CFE+=,,6+B,8,CB,,

¿Puedes identificar las diferentes partes del formato?

El genoma de referencia es el del SARS-CoV-2, i es troba en l'arxiu `data/referencia.fna`. Aquestes són les seues primeres línies:

    >NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
    ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
    CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
    TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
    TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
    CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
    GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
    CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT

¿Reconoces el formato?

"Mapear" es encontrar la posición del genoma de referencia donde cada secuencia encuentra su mejor alineamiento. El mapeo

In [1]:
library(ShortRead)
library(Biostrings)


Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: BiocParallel

Loading required package: Biostrings

Loading required package: S4Ve